In [ ]:
#This Model detects just the cells from a whole panel image

#Step 1. load dataset from roboflow 

# ✅ Install required packages
!pip install roboflow ultralytics

# ✅ Import necessary modules
from roboflow import Roboflow
from ultralytics import YOLO
import os
import glob

# ✅ Download dataset from Roboflow (YOLOv8 format)
rf = Roboflow(api_key="TYCxQVZrrdnE0E1tmaSw")
project = rf.workspace("sa-tvpln").project("cell-detector-ackxk")
version = project.version(1)
dataset = version.download("yolov8")


In [ ]:
#Step 2 Convert any Polygon Anotation to bounding Box
# keep getting very low accuracy when I dont convert the polygons

# ✅ Function to convert polygon to bounding box
def polygon_to_bbox(coords):
    xs = coords[::2]
    ys = coords[1::2]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    return [x_center, y_center, width, height]

# ✅ Convert all label .txt files (train, valid, test) to bounding boxes
def convert_all_labels(base_dir):
    label_paths = [
        os.path.join(base_dir, "train", "labels"),
        os.path.join(base_dir, "valid", "labels"),
        os.path.join(base_dir, "test", "labels")
    ]

    for label_dir in label_paths:
        if not os.path.exists(label_dir):
            continue
        label_files = glob.glob(os.path.join(label_dir, "*.txt"))

        for file in label_files:
            with open(file, "r") as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = list(map(float, line.strip().split()))
                class_id = int(parts[0])
                coords = parts[1:]
                if len(coords) > 4:  # polygon
                    bbox = polygon_to_bbox(coords)
                    new_line = f"{class_id} {' '.join(map(str, bbox))}\n"
                    new_lines.append(new_line)
                else:  # already bbox
                    new_lines.append(line)

            with open(file, "w") as f:
                f.writelines(new_lines)

# ✅ Run the conversion
convert_all_labels(dataset.location)

# ✅ Define path to the dataset config file (data.yaml)
DATA_YAML_PATH = os.path.join(dataset.location, "data.yaml")



In [ ]:
#Step 3 Model Training
model = YOLO("yolov8s.pt")  

# ✅ Train the model
results = model.train(
    data=DATA_YAML_PATH,
    epochs=100,
    imgsz=640,
    batch=32,
    project="cell_detector_project",
    name="yolov8_cell_detector",
    patience=30,
    optimizer="SGD",
    cos_lr=True,
    fliplr=0.5,
    flipud=0.2,
    mosaic=1.0,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=5.0,
    scale=0.5,
    shear=2.0,
    perspective=0.0005,
    verbose=True
)